# Analizamos cómo el modelo representa aspectos de la distribución de una variable

En este tutorial vamos a explorar como es la distribución de probabilidades en el modelo para un punto:
- Vamos a seleccionar un punto
- Tomar datos diarios correspondiende a la semana 2
- Calcular la función de distribución de probabilidad y la función de distribución de probabilidad acumulada
En este caso vamos a trabajar con la variable precipitación

In [1]:
# Partimos importando los modulos a utilizar:
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import xarray as xr
import s3fs
%matplotlib inline


Primero definimos algunas funciones que vamos a usar

Vamos a estudiar la distribucion para un punto, con la variable lluvia para las fechas alrededor del día 5/2/2018.

In [2]:
variable = 'rain'
lat = -29.18
lon = -58.07
ref_date = datetime.datetime.strptime('20180502', "%Y%m%d")

In [3]:
# funcion para acomodar xarrays antes de concatenarlos
def preprocess_ds(ds, variable=variable):
    ds = ds.assign_coords(S=ds.time.values[0], M=int(ds[variable].encoding['source'][-6:-4]))
    ds['time'] = ds.time.values - ds.time.values[0]
    ds = ds.rename({'time': 'leadtime'})
    return ds

# funcion para calcular la CDF de los datos
def calcular_cdf(datos):
    n = len(datos)
    x_ordenados = np.sort(datos)
    cdf = np.arange(1, n + 1) / n
    return x_ordenados, cdf

# función para obtener el segundo dato de cada grupo
def get_second_data(group):
    return group.isel(S=1)

Vamos a tomar de la base de datos de sissa los datos corregidos. Vamos a ver como se programa para seleccionar todas las fechas alrededor del 2 de mayo de cada año del período 2010-2019.

In [ ]:
# DATOS CORREGIDOS
fs = s3fs.S3FileSystem(anon=True)
BUCKET_NAME = 'sissa-forecast-database'
tforecast = 'subseasonal'
modelo = 'GEFSv12_corr'
list_df = []
for i in np.arange(2010, 2020, 1): #loop sobre años
    loop_date = ref_date.replace(year=i) - timedelta(days=15) # reemplazo el año y arranco 15 días antes de la fecha
    while loop_date < (ref_date.replace(year=i) + timedelta(days=15)): # el loop es hasta 15 días despues
        if loop_date.weekday() == 2: # los pronos están disponibles los miercoles
            PATH = tforecast + '/' + modelo + '/' + variable + '/' + loop_date.strftime('%Y') + '/' + loop_date.strftime('%Y%m%d') + '/'            
            # Listamos todos los archivos dentro del bucket + PATH
            remote_files = fs.glob('s3://' + BUCKET_NAME + '/' + PATH + '*.nc')
            # Iterate through remote_files to create a fileset
            fileset = [fs.open(file) for file in remote_files]
            # This works
            data = xr.open_mfdataset(fileset, combine='nested', concat_dim='M', preprocess=preprocess_ds)
            data = data.sel(lat=lat, lon=lon, method='nearest')
            list_df.append(data)
            
        loop_date += timedelta(days=1)

ds_corr = xr.concat(list_df, dim='S')
print(ds_corr)

Repetimos para los datos sin corregir

In [ ]:
# DATOS SIN CORREGIR
fs = s3fs.S3FileSystem(anon=True)
BUCKET_NAME = 'sissa-forecast-database'
tforecast = 'subseasonal'
modelo = 'GEFSv12'
list_df = []
for i in np.arange(2010, 2020, 1): #loop sobre años
    loop_date = ref_date.replace(year=i) - timedelta(days=15) # reemplazo el año y arranco 15 días antes de la fecha
    while loop_date < (ref_date.replace(year=i) + timedelta(days=15)): # el loop es hasta 15 días despues
        if loop_date.weekday() == 2: # los pronos están disponibles los miercoles
            PATH = tforecast + '/' + modelo + '/' + variable + '/' + loop_date.strftime('%Y') + '/' + loop_date.strftime('%Y%m%d') + '/'            
            # Listamos todos los archivos dentro del bucket + PATH
            remote_files = fs.glob('s3://' + BUCKET_NAME + '/' + PATH + '*.nc')
            # Iterate through remote_files to create a fileset
            fileset = [fs.open(file) for file in remote_files]
            # This works
            data = xr.open_mfdataset(fileset, combine='nested', concat_dim='M', preprocess=preprocess_ds)
            data = data.sel(lat=lat, lon=lon, method='nearest')
            list_df.append(data)
            
        loop_date += timedelta(days=1)

ds_uncal = xr.concat(list_df, dim='S')

Abrimos los datos observados y selecciono la segunda semana a patir de los miercoles alrededor de la fecha de interes para los años de estudio

In [ ]:
# Obtenemos los datos de ERA5
fs = s3fs.S3FileSystem(anon=True)
BUCKET_NAME = 'sissa-forecast-database'
tforecast = 'ERA5'
modelo = 'ERA5'
list_df = []
PATH = tforecast + '/' + variable 
remote_files = fs.glob('s3://' + BUCKET_NAME + '/' + PATH + '/' + '*.nc')
# Iterate through remote_files to create a fileset
fileset = [fs.open(file) for file in remote_files]
data = xr.open_mfdataset(fileset, combine='nested', concat_dim='time')
data = data.sel(latitude=lat, longitude=lon, method='nearest')

for i in np.arange(2010, 2020, 1):
    loop_date = ref_date.replace(year=i) - timedelta(days=15)
    while loop_date < (ref_date.replace(year=i) + timedelta(days=15)):
        if loop_date.weekday() == 2:
            era = data.sel(time= slice(loop_date + timedelta(days=8), loop_date + timedelta(days=14)))
            era['S'] = loop_date
            era['time'] = era['time'] - era['S']
            list_df.append(era)
        
        loop_date += timedelta(days=1)
ds_era = xr.concat(list_df, dim='S')
ds_era = ds_era.compute()

In [ ]:
print(ds_corr, ds_uncal, ds_era)

Selecciono la semana 2

In [ ]:
# selecciono semana 2 (8-14 dias)
ds_corr_w2 = ds_corr.sel(leadtime=slice(np.timedelta64(8, 'D'), np.timedelta64(14, 'D'))).compute()
print(ds_corr_w2)

Repito para los datos corregidos

In [ ]:
# selecciono semana 2 (8-14 dias)
ds_uncal_w2 = ds_uncal.sel(leadtime=slice(np.timedelta64(8, 'D'), np.timedelta64(14, 'D'))).compute()
print(ds_uncal_w2)

In [ ]:
# Ajuste de los datos a una distribución normal
aux_uncal =ds_uncal_w2[variable].values.flat
aux_corr =ds_corr_w2[variable].values.flat
aux_era = ds_era[variable].values.flat[~np.isnan(ds_era[variable].values.flat)] 

bins = np.arange(0, 60, 5)

plt.hist(aux_uncal, bins, density=True, histtype='step', color='r', alpha=0.5, label='GEFS No-corregida', linewidth=1.5)
plt.hist(aux_corr, bins, density=True, histtype='step', color='b', alpha=0.5, label='GEFS Corregida', linewidth=1.5)
plt.hist(aux_era, bins, density=True, histtype='step', color='k', alpha=0.5, label='ERA5', linewidth=1.5)


plt.xlabel(variable)
plt.ylabel('Densidad de probabilidad')
plt.title('Ajuste de datos a una distribución normal')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x_ordenados1, cdf1 = calcular_cdf(aux_corr)
x_ordenados2, cdf2 = calcular_cdf(aux_uncal)
x_ordenados3, cdf3 = calcular_cdf(aux_era)

# Plotear la CDF
plt.plot(x_ordenados1, cdf1, 'b', label='Corregidos')
plt.plot(x_ordenados2, cdf2, 'r', label='Sin corregir')
plt.plot(x_ordenados3, cdf3, 'k', label='ERA5')
plt.xlabel(variable)
plt.ylabel('Probabilidad acumulada')
plt.title('Función de Distribución Acumulativa (CDF)')
plt.legend()
plt.grid(True)
plt.show()


Ahora miramos la evolución de la lluvia acumulada para el segundo miercoles de mayo del registro. Vamos a ver cómo seleccionar los pronosticos inicializados esa segunda semana

In [ ]:
#seleccionamos el me de mayo
ds_corr_w2 = ds_corr_w2.sum('leadtime', skipna=True).isel(S= ds_uncal_w2.S.dt.month==5)
ds_uncal_w2 = ds_uncal_w2.sum('leadtime', skipna=True).isel(S= ds_uncal_w2.S.dt.month==5)
ds_era = ds_era.sum('time', skipna=True).isel(S= ds_era.S.dt.month==5)


In [ ]:
print(ds_corr_w2, ds_uncal_w2, ds_era)

In [ ]:
# Selecciona el segundo miércoles de cada mes
ds_corr_w2_smier = ds_corr_w2.groupby('S.year').apply(get_second_data)
ds_uncal_w2_smier = ds_uncal_w2.groupby('S.year').apply(get_second_data)
ds_era_w2_smier = ds_era.groupby('S.year').apply(get_second_data)


In [ ]:
fig = plt.figure()
ds_uncal_w2_smier[variable].plot.scatter(x='year',color='gray', alpha=0.5, label='sin corregir')
ds_era_w2_smier[variable].plot.scatter(x='year', color='k', label='ERA 5')
plt.axis([2009, 2020, 0, 200])
plt.legend()
plt.savefig(variable + '_sin_corregir.jpg', dpi=200)

In [ ]:
fig = plt.figure()
ds_corr_w2_smier[variable].plot.scatter(x='year', color='orange', alpha=0.5, label='corregido')
ds_era_w2_smier[variable].plot.scatter(x='year', color='k', label='ERA 5')
plt.axis([2009, 2020, 0, 200])
plt.legend()
plt.savefig(variable + '_corregido.jpg', dpi=200)